In [ ]:
# I was getting a ModuleNotFoundError, and I discovered that the cwd (current working directory) was
#   '<Github Home>/market-modeling/examples/' but it needs to be '<Github Home>/market-modeling/'
# The following code will add the cwd's parent directory to the path Python uses to search for packages
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import os
from datetime import datetime

from gbm_models.Models import IndependentModel
from markets.Markets import BaseMarket
from assets.Assets import BaseAsset
from metrics.Metrics import VarType, TimeseriesMetric

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DEBUG = True
DATA_DIR = '../data'
MARKET_NAME = 'popular_stocks'
TS_DIR = 'growth_timeseries'

In [ ]:
asset_list = []
asset_metadata = pd.read_csv(os.path.join(DATA_DIR, MARKET_NAME, 'asset_metadata.tsv'), sep='\t')

for i, row in asset_metadata.iterrows():
  if DEBUG:
    if i == 5:
      break
  
  asset_ts = pd.read_csv(
    os.path.join(DATA_DIR, MARKET_NAME, TS_DIR, f"{row['id']}.csv"),
    parse_dates=['date'],
    dtype={'price' : np.float32}
  )
  
  asset = BaseAsset(
    row['name'],
    row['id'],
    {
      'signal': TimeseriesMetric(asset_ts['date'], asset_ts['price'], VarType.QUANTITATIVE)
    }
  )
  asset_list.append(asset)
  
market = BaseMarket(asset_list, MARKET_NAME)
indep_model = IndependentModel(market)

In [ ]:
market.align_timeseries()
indep_model.estimate_parameters()

In [ ]:
df_results = pd.DataFrame({
  'id' : [asset.asset_id for asset in indep_model.market.assets],
  'drifts' : indep_model.drifts,
  'vols' : indep_model.vols
})

df_results

In [ ]:
sim_dates = pd.to_datetime(
  pd.Series(['12-16-2024', '12-18-2024', '12-20-2024', '12-23-2024'], name='dates'),
  format='%m-%d-%Y'
)

sim_res = indep_model.simulate(sim_dates, num_sims=20)
sim_counterfact = indep_model.simulate(sim_dates, 1, add_BM=False)

In [ ]:
colors = ['C0', 'C1', 'C2', 'C3', 'C4']
fig, ax = plt.subplots(figsize=(10,8))

for i in range(len(colors)): #range(indep_model.N_ASSETS):
  asset_i = indep_model.market.assets[i % indep_model.N_ASSETS]
  disp_name = asset_i.asset_id
  ax.plot(asset_i.ts_df['time'], asset_i.ts_df['signal'], label=f'{disp_name} true signal',
           color=colors[i % indep_model.N_ASSETS])
  
  fut_dates = pd.concat((asset_i.ts_df['time'].iloc[-1:], sim_dates))
  ax.set_xlim([datetime(2024, 11, 1), sim_dates.max()])
  ax.plot(fut_dates, sim_counterfact[0, i, :], label=f'{disp_name} avg prediction',
           color=colors[i % indep_model.N_ASSETS], linestyle='dashed')
  for j_sim in range(sim_res.shape[0]):
    ax.plot(fut_dates, sim_res[j_sim, i, :],
            alpha=0.5, color=colors[i % indep_model.N_ASSETS], linestyle='dotted')
ax.set(xlabel='Date', ylabel='close price', title='Close Prices vs Time')
fig.tight_layout()
fig.legend(loc=2);